In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from abc import ABC, abstractclassmethod, abstractmethod
from typing import Callable, List
import requests 
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

pd.set_option("display.max_rows",4999)



url = ("https://www.thebalance.com/u-s-inflation-rate-history-by-year-and-forecast-3306093")


class FederalRates(ABC):
    
    def __init__(self, fundRate, inflation):
        try: 
            self.fundRate = fundRate
            self.inflation = inflation
            super(FederalRates, self).__init__()
        except AttributeError as e:
            print(e)
            
    @abstractmethod
    def execute(self):
        return self.__init__()   
    
    #Defining and identifying the variables from source file.
    #Important variables YEAR - MM - DD , Effective Federal Funds Rate
    @classmethod
    def readData(cls, path):
        rawData = pd.read_csv(path)
        return (rawData)

    @classmethod
    def getURL(cls, url):
        datasets = []
        try:
            raw = requests.get(url)
            if raw.status_code == 200:
                soup = BeautifulSoup(raw.content)
                table = soup.find_all('table')[0]
                for row in table.find_all('tr'):
                    columns = row.find_all('td')
                    for column in columns:
                        datasets.append(column.get_text())
                return pd.DataFrame(datasets)
            else:
                raw.status_code
        except requests.exceptions.HTTPError as e:
            print(e)
            
    
    
class InterestRate(FederalRates):
    def execute(self):
        a = self.fundRate
        b = self.inflation
        print(b)    

    
def main(rawInterestRateData, rawInflationRateData):
    interestRate = InterestRate(FederalRates.readData(rawInterestRateData), FederalRates.readData(rawInflationRateData))
    return interestRate.execute()
 
if __name__ == "__main__":
    main("./FF.csv", "./index.csv")


     Year  Month  Day  Federal Funds Target Rate  Federal Funds Upper Target  \
0    1954      7    1                        NaN                         NaN   
1    1954      8    1                        NaN                         NaN   
2    1954      9    1                        NaN                         NaN   
3    1954     10    1                        NaN                         NaN   
4    1954     11    1                        NaN                         NaN   
5    1954     12    1                        NaN                         NaN   
6    1955      1    1                        NaN                         NaN   
7    1955      2    1                        NaN                         NaN   
8    1955      3    1                        NaN                         NaN   
9    1955      4    1                        NaN                         NaN   
10   1955      5    1                        NaN                         NaN   
11   1955      6    1                   